In [1]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

# changed model


### regression

In [1]:
import numpy as np
dataset_path = "dataset"
# 加载数据
X_train = np.load(f'{dataset_path}/X_train.npy')
y_train = np.load(f'{dataset_path}/y_train.npy').astype(int).flatten()
X_val = np.load(f'{dataset_path}/X_val.npy')
y_val = np.load(f'{dataset_path}/y_val.npy').astype(int).flatten()
X_test = np.load(f'{dataset_path}/X_test.npy')
y_test = np.load(f'{dataset_path}/y_test.npy').astype(int).flatten()


In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

 #将数据转换为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 数据归一化
mean = X_train_tensor.mean(dim=0, keepdim=True)
std = X_train_tensor.std(dim=0, keepdim=True)
X_train_tensor = (X_train_tensor - mean) / std
X_val_tensor = (X_val_tensor - mean) / std
X_test_tensor = (X_test_tensor - mean) / std

# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 定义 Logistic Regression 模型
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        outputs = self.linear(x)
        return outputs

model = LogisticRegressionModel(input_dim=12, num_classes=11)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10000
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
    
    val_loss /= len(val_loader.dataset)
    print(f'Validation Loss: {val_loss:.4f}')

# 测试模型
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = correct / total
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}')


Epoch [1/10000], Loss: 2.4688
Validation Loss: 2.0977
Epoch [2/10000], Loss: 1.9532
Validation Loss: 1.8474
Epoch [3/10000], Loss: 1.7969
Validation Loss: 1.7637
Epoch [4/10000], Loss: 1.7325
Validation Loss: 1.7175
Epoch [5/10000], Loss: 1.6930
Validation Loss: 1.6875
Epoch [6/10000], Loss: 1.6633
Validation Loss: 1.6625
Epoch [7/10000], Loss: 1.6385
Validation Loss: 1.6413
Epoch [8/10000], Loss: 1.6174
Validation Loss: 1.6217
Epoch [9/10000], Loss: 1.5990
Validation Loss: 1.6046
Epoch [10/10000], Loss: 1.5821
Validation Loss: 1.5914
Epoch [11/10000], Loss: 1.5675
Validation Loss: 1.5777
Epoch [12/10000], Loss: 1.5540
Validation Loss: 1.5647
Epoch [13/10000], Loss: 1.5417
Validation Loss: 1.5564
Epoch [14/10000], Loss: 1.5305
Validation Loss: 1.5441
Epoch [15/10000], Loss: 1.5204
Validation Loss: 1.5368
Epoch [16/10000], Loss: 1.5113
Validation Loss: 1.5256
Epoch [17/10000], Loss: 1.5022
Validation Loss: 1.5186
Epoch [18/10000], Loss: 1.4941
Validation Loss: 1.5081
Epoch [19/10000], L

### efficientnet

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from efficientnet_pytorch import EfficientNet



# 将数据转换为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 数据归一化
mean = X_train_tensor.mean(dim=0, keepdim=True)
std = X_train_tensor.std(dim=0, keepdim=True)
X_train_tensor = (X_train_tensor - mean) / std
X_val_tensor = (X_val_tensor - mean) / std
X_test_tensor = (X_test_tensor - mean) / std

# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 定义模型
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = SimpleMLP(input_dim=12, num_classes=11)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
    
    val_loss /= len(val_loader.dataset)
    print(f'Validation Loss: {val_loss:.4f}')

# 测试模型
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = correct / total
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}')


Epoch [1/100], Loss: 1.6591
Validation Loss: 1.4985
Epoch [2/100], Loss: 1.3953
Validation Loss: 1.3730
Epoch [3/100], Loss: 1.3060
Validation Loss: 1.3076
Epoch [4/100], Loss: 1.2610
Validation Loss: 1.2865
Epoch [5/100], Loss: 1.2279
Validation Loss: 1.2484
Epoch [6/100], Loss: 1.2007
Validation Loss: 1.2542
Epoch [7/100], Loss: 1.1818
Validation Loss: 1.2360
Epoch [8/100], Loss: 1.1666
Validation Loss: 1.2196
Epoch [9/100], Loss: 1.1527
Validation Loss: 1.1946
Epoch [10/100], Loss: 1.1389
Validation Loss: 1.1912
Epoch [11/100], Loss: 1.1329
Validation Loss: 1.1781
Epoch [12/100], Loss: 1.1120
Validation Loss: 1.1606
Epoch [13/100], Loss: 1.1025
Validation Loss: 1.1563
Epoch [14/100], Loss: 1.0880
Validation Loss: 1.1702
Epoch [15/100], Loss: 1.0833
Validation Loss: 1.1388
Epoch [16/100], Loss: 1.0734
Validation Loss: 1.1243
Epoch [17/100], Loss: 1.0617
Validation Loss: 1.1392
Epoch [18/100], Loss: 1.0488
Validation Loss: 1.1313
Epoch [19/100], Loss: 1.0437
Validation Loss: 1.1149
Ep

# original model

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models

# 定义训练数据和标签
train_data = torch.randn(100, 1, 4, 4)  # 假设有100个样本，每个样本是1x4x4的矩阵
train_labels = torch.randint(0, 10, (100,))  # 100个标签，每个标签是0~9之间的整数

# 图像预处理（这里不需要 resize 和 crop，因为输入已经是4x4）
preprocess = transforms.Compose([
    transforms.Normalize(mean=[0.5], std=[0.5])  # 假设数据在0~1之间
])

# 加载预训练的 ResNet-18 模型
model = models.resnet18(pretrained=True)

# 调整模型的第一层和最后一层
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # 调整第一层以适应1通道输入
model.fc = nn.Linear(model.fc.in_features, 10)  # 调整最后一层以适应10分类

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    inputs = preprocess(train_data)
    outputs = model(inputs)
    
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [4]:
import timm
import torch
import torch.nn as nn
import torchvision.transforms as transforms

# 定义训练数据和标签
train_data = torch.randn(100, 1, 4, 4)  # 假设有100个样本，每个样本是1x4x4的矩阵
train_labels = torch.randint(0, 10, (100,))  # 100个标签，每个标签是0~9之间的整数

# 图像预处理（这里不需要 resize 和 crop，因为输入已经是4x4）
preprocess = transforms.Compose([
    transforms.Normalize(mean=[0.5], std=[0.5])  # 假设数据在0~1之间
])

# 加载 EfficientNet-B0 模型
model = timm.create_model('efficientnet_b0', pretrained=True, in_chans=1, num_classes=10)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    inputs = preprocess(train_data)
    outputs = model(inputs)
    
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


ModuleNotFoundError: No module named 'timm'

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models

# 定义训练数据和标签
train_data = torch.randn(100, 1, 4, 4)  # 假设有100个样本，每个样本是1x4x4的矩阵
train_labels = torch.randn(100)  # 100个回归标签

# 图像预处理（这里不需要 resize 和 crop，因为输入已经是4x4）
preprocess = transforms.Compose([
    transforms.Normalize(mean=[0.5], std=[0.5])  # 假设数据在0~1之间
])

# 加载预训练的 ResNet-18 模型
model = models.resnet18(pretrained=True)

# 调整模型的第一层和最后一层
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # 调整第一层以适应1通道输入
model.fc = nn.Linear(model.fc.in_features, 1)  # 调整最后一层为回归任务

# 损失函数和优化器
criterion = nn.MSELoss()  # 使用均方误差损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    inputs = preprocess(train_data)
    outputs = model(inputs)
    
    loss = criterion(outputs.squeeze(), train_labels)
    loss.backward()
    optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
import timm
import torch
import torch.nn as nn
import torchvision.transforms as transforms

# 定义训练数据和标签
train_data = torch.randn(100, 1, 4, 4)  # 假设有100个样本，每个样本是1x4x4的矩阵
train_labels = torch.randn(100)  # 100个回归标签

# 图像预处理（这里不需要 resize 和 crop，因为输入已经是4x4）
preprocess = transforms.Compose([
    transforms.Normalize(mean=[0.5], std=[0.5])  # 假设数据在0~1之间
])

# 加载 EfficientNet-B0 模型
model = timm.create_model('efficientnet_b0', pretrained=True, in_chans=1, num_classes=1)

# 损失函数和优化器
criterion = nn.MSELoss()  # 使用均方误差损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    inputs = preprocess(train_data)
    outputs = model(inputs)
    
    loss = criterion(outputs.squeeze(), train_labels)
    loss.backward()
    optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
